# 유튜브 랭킹 데이터 수집하기

In [19]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 

In [21]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(r"D:\Workspace\chromedriver.exe", options=options)

In [22]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [23]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [24]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [25]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [26]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [27]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [28]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6410만', '190억1809만', '371')

In [29]:
channels = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category,name,subscriber,view,video])

In [30]:
df = pd.DataFrame(channels, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6410만,190억1809만,371
1,음악/댄스/가수,HYBE LABELS,6040만,187억3584만,654
2,음악/댄스/가수,BANGTANTV,5650만,122억1297만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억9538만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [31]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6621만,433
97,음악/댄스/가수,NCT DREAM,387만,3억7615만,229
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,취미/라이프,JaeYeol ASMR 재열,380만,10억1154만,970


- 만과 억을 숫자로 바꿔주는 함수

In [32]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [33]:
convert_unit('123억6,557만개')

'12,365,570,000'

- 두번째 페이지 - XPath로 찾아 Click() 하여 이동하기

In [34]:
# //*[@id="list-skin"]/nav/span/a[1]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [35]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [36]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [37]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [38]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,735,840,000",654
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,895,380,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


## page 1에서 10까지 크롤링하기

In [39]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [40]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,735,840,000",654
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,895,380,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [41]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,게임,OGN,"510,000","608,880,000","27,433"
996,미분류,밉지않은 관종언니,"510,000","67,950,000",170
997,게임,미소,"510,000","210,780,000","3,084"
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


In [42]:
df.to_csv('유튜브_순위.csv', index=None)

In [43]:
driver.close()